test:
1. get embedding
2. get discriminator
3. get off-manifolder
4. get jacobian of the chain
5. visualize jacobians

In [152]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../src/')
# from diffusion import DiffusionModel
# from evaluate import get_results
from omegaconf import OmegaConf
# from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch
from model2 import Autoencoder, Preprocessor, Discriminator
import magic
import torch
import pathlib
import copy

import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
from omegaconf import OmegaConf
import os
import glob
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from geodesic import jacobian, velocity, CondCurve, GeodesicBridgeOverfit, GeodesicBridge
from plotly3d.plot import scatter, trajectories
import torch
from torch import nn
import pytorch_lightning as pl
from procrustes import Procrustes
from torch.utils.data import DataLoader, TensorDataset

In [32]:
main_sweep_id = '59muwbsf'
main_disc_sweep_id = 'ywep3ixr'
main_data_name = 'hemisphere_none_0'
main_weights_cycle = 1.
main_dimensions_latent = 2

wandb.login()
api = wandb.Api()

entity = "xingzhis"
project = "dmae"
sweep_id = main_sweep_id
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
# Initialize an empty list to store run data
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

data_name = main_data_name
# run_ids = df[(df['data.name'] == data_name) & (df['cfg/loss/weights/cycle'] == 1.) & (df['cfg/dimensions/latent'] == 3)]['run_id']
run_ids = df[(df['data.name'] == data_name) & (df['loss.weights.cycle'] == main_weights_cycle) & (df['dimensions.latent'] == main_dimensions_latent)]['run_id']
assert len(run_ids) == 1
run_id = run_ids.iloc[0]
run = api.run(f"{entity}/{project}/{run_id}")
# run = api.run(f"{entity}/{project}/{run_id}")
folder_path = '../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
# cfg.data.root = '../' + cfg.data.root
model = Autoencoder.load_from_checkpoint(ckpt_path)
data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)
with torch.no_grad():
    model.eval()
    x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
    z = model.encoder(x)
    xh = model.decoder(z)
    zhh = model.encoder(xh)
zc = z.cpu().numpy()
# fig = scatter(zc, s=2, alpha=0.2, title='latent', filename=f'{results_path}/latent.html')
# wandb.log({"latent": wandb.Plotly(fig)})
# fig = scatter(zhh, s=2, alpha=0.2, title='latent_reembedd', filename=f'{results_path}/latent_reembedded.html')
# wandb.log({"latent_reembedded": wandb.Plotly(fig)})
# fig = scatter(x, s=2, alpha=0.2, title='data', filename=f'{results_path}/data.html')
# wandb.log({"data": wandb.Plotly(fig)})
# fig = scatter(xh, s=2, alpha=0.2, title='reconstruction', filename=f'{results_path}/reconstruction.html')
# wandb.log({"reconstruction": wandb.Plotly(fig)})

sweep_id = main_disc_sweep_id
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
# Initialize an empty list to store run data
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)
run_ids = df[(df['data.name'] == data_name)][['run_id']]
assert len(run_ids) == 1
run_id = run_ids.iloc[0]
run = api.run(f"{entity}/{project}/{run_ids.iloc[0].values[0]}")
folder_path = '../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
# cfg.data.root = '../' + cfg.data.root
data_ = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)
discriminator = Discriminator.load_from_checkpoint(ckpt_path)
with torch.no_grad():
    discriminator.eval()
    x_disc = torch.tensor(data_['data'], dtype=torch.float32, device=discriminator.device)
    probab = discriminator.positive_proba(x_disc)
    z_disc = model.encoder(x_disc)

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.
  rank_zero_warn(
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.
  rank_zero_warn(


In [98]:
def offmanifolder_maker(
    encoder, 
    discriminator, 
    emb_dim,
    device,
    folding_dim = 10,
    density_exponential = 4,
):
    assert folding_dim > emb_dim
    # preserve_matrix = torch.zeros(emb_dim, folding_dim, dtype=torch.float).to(device)
    # random_matrix = torch.randn(1, folding_dim - emb_dim).to(device)
    random_matrix = torch.ones(1, folding_dim - emb_dim, dtype=torch.float, device=device)
    def ofm_(x):
        z = encoder(x)
        # preserved_subspace = z @ preserve_matrix
        preserved_subspace = torch.cat([z, torch.zeros((z.size(0), folding_dim-emb_dim), dtype=z.dtype, device=device)], dim=1)
        random_dirs = torch.cat([torch.zeros((z.size(0), emb_dim), dtype=z.dtype, device=device), random_matrix.repeat(z.size(0), 1)], dim=1)
        # weighting_factor = torch.exp((1-discriminator(x))*density_exponential) - 1 # starts at 1; gets higher immediately.
        weighting_factor = ((1-discriminator(x))*density_exponential) # starts at 1; gets higher immediately.
        return preserved_subspace + random_dirs*weighting_factor[:,None]
    return ofm_


In [141]:
def offmanifolder1_maker(
    encoder_, 
    discriminator_, 
    disc_factor = 4,
):
    def ofm_(x):
        z = encoder_(x)
        weighting_factor = discriminator_(x)*disc_factor
        return torch.cat([z, weighting_factor.unsqueeze(1).repeat(1, 1)], dim=1)
    return ofm_

In [137]:
disc_func = lambda x: discriminator(x)[:,0]
ofm = offmanifolder_maker(model.encoder, discriminator.positive_proba, main_dimensions_latent, model.device, folding_dim=3, density_exponential=1)

In [143]:
ofm_emb = ofm(x_disc).detach()
logit = discriminator(x)[:,0].detach()
scatter(ofm_emb, logit)

In [149]:
disc_func = lambda x: 1 - discriminator.positive_proba(x)
ofm = offmanifolder1_maker(model.encoder, disc_func, disc_factor=1)

In [151]:
ofm_emb = ofm(x_disc).detach()
# logit = discriminator(x)[:,0].detach()
scatter(ofm_emb, probab)